In [ ]:
!pip install pennylane
!pip install noisyopt
!pip install ipympl
!pip install pennylane-qiskit

In [ ]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

from pennylane import qchem
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from noisyopt import minimizeSPSA
import time

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [ ]:
# VQE parameters
# The number of iterations of the optimizer
niter_spsa = 200

# The true energy of the H2 molecule
true_energy = -1.136189454088

# Seed number, for reproducibility of random initialization
seednum=0

# Optimizer hyperparameters
c_value=0.3
a_value=1.5

# Setup the data of the molecule to optimize for
symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])
h2_ham, num_qubits = qchem.molecular_hamiltonian(symbols, coordinates)

In [ ]:
# The classical simulation, no noise
# Variational ansatz for H2 
def circuit(params, wires): 
    qml.BasisState(np.array([1, 1, 0, 0]), wires=wires)
    for i in wires:
        qml.Rot(*params[i], wires=i)
    qml.CNOT(wires=[2,3])
    qml.CNOT(wires=[2,0])
    qml.CNOT(wires=[3,1])
    
num_params=3 
devicename= "default.qubit"
dev = qml.device(devicename, wires=num_qubits)

def exp_val_circuit(params):
    circuit(params, range(dev.num_wires))
    return qml.expval(h2_ham)

# Pennylane uses Quantum Nodes (Qnodes)
cost_spsa = qml.QNode(exp_val_circuit, dev)

# Wrapping the cost function and flattening the parameters to be compatible with noisyopt which assumes a flat array of input parameters
def wrapped_cost(params):  
    return cost_spsa(params.reshape(num_qubits, num_params))

def callback_fn(xk):
    cost_val = wrapped_cost(xk)
    simulation_energies.append(cost_val)

## Set initial rotation parameters
np.random.seed(seednum)
init_params = np.random.normal(0, np.pi, (num_qubits, 3), requires_grad=True)
initial_name='random seed '+str(seednum)
params = init_params.copy().reshape(num_qubits * num_params)
simulation_energies = [wrapped_cost(params)]

res = minimizeSPSA(wrapped_cost,
        x0=params,
        niter=niter_spsa,
        paired=False,
        c=c_value,
        a=a_value,
        disp=True,
        callback=callback_fn)

print("Result:", res)
print()
print(f"Final estimated value of the ground-state energy = {simulation_energies[-1]:.8f} Ha")
print(
   f"Accuracy with respect to the true energy: {np.abs(simulation_energies[-1] - true_energy):.8f} Ha"
)

In [ ]:
# Simulating random noise
# Variational ansatz for H2 
def circuit(params, wires): 
    qml.BasisState(np.array([1, 1, 0, 0]), wires=wires)
    for i in wires:
        qml.Rot(*params[i], wires=i)
    qml.CNOT(wires=[2,3])
    qml.CNOT(wires=[2,0])
    qml.CNOT(wires=[3,1])
    
    # Random Noise
    rand_angle = np.pi + np.random.rand()
    # np.random.rand() uniformly samples from [0, 1)
    for i in wires:
        qml.RX(rand_angle, wires=0)

devicename= "default.qubit"
dev = qml.device(devicename, wires=num_qubits)#, noise_model = my_noise_model)

def exp_val_circuit(params):
    circuit(params, range(dev.num_wires))
    return qml.expval(h2_ham)

# Pennylane uses Quantum Nodes (Qnodes)
cost_spsa = qml.QNode(exp_val_circuit, dev)

# Wrapping the cost function and flattening the parameters to be compatible with noisyopt which assumes a flat array of input parameters
def wrapped_cost(params):  
    return cost_spsa(params.reshape(num_qubits, num_params))

def callback_fn(xk):
    cost_val = wrapped_cost(xk)
    random_noise_results.append(cost_val)

## Set initial rotation parameters
np.random.seed(seednum)
init_params = np.random.normal(0, np.pi, (num_qubits, 3), requires_grad=True)
initial_name='random seed '+str(seednum)
params = init_params.copy().reshape(num_qubits * num_params)
random_noise_results = [wrapped_cost(params)]

res = minimizeSPSA(wrapped_cost,
        x0=params,
        niter=niter_spsa,
        paired=False,
        c=c_value,
        a=a_value,
        disp=True,
        callback=callback_fn)

print("Result:", res)
print()
print(f"Final estimated value of the ground-state energy = {random_noise_results[-1]:.8f} Ha")
print(
   f"Accuracy with respect to the true energy: {np.abs(random_noise_results[-1] - true_energy):.8f} Ha"
)

In [ ]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import depolarizing_error
import qiskit
import qiskit.providers.aer.noise as noise

my_noise_model = NoiseModel()
error = depolarizing_error(0.05, 2)
my_noise_model.add_all_qubit_quantum_error(error, ['cx'])

dev = qml.device('qiskit.aer', wires=num_qubits, noise_model = my_noise_model)

def exp_val_circuit(params):
    circuit(params, range(dev.num_wires))
    return qml.expval(h2_ham)

# Pennylane uses Quantum Nodes (Qnodes)
cost_spsa = qml.QNode(exp_val_circuit, dev)

# Wrapping the cost function and flattening the parameters to be compatible with noisyopt which assumes a flat array of input parameters
def wrapped_cost(params):  
    return cost_spsa(params.reshape(num_qubits, num_params))

def callback_fn(xk):
    cost_val = wrapped_cost(xk)
    cx_noise.append(cost_val)
    
## Set initial rotation parameters
np.random.seed(seednum)
init_params = np.random.normal(0, np.pi, (num_qubits, 3), requires_grad=True)
initial_name='random seed '+str(seednum)
params = init_params.copy().reshape(num_qubits * num_params) #flatten
cx_noise = [wrapped_cost(params)]

res = minimizeSPSA(wrapped_cost,
        x0=params,
        niter=niter_spsa,
        paired=False,
        c=c_value,
        a=a_value,
        disp=True,
        callback=callback_fn)

print("Result:", res)
print()
print(f"Final estimated value of the ground-state energy = {cx_noise[-1]:.8f} Ha")
print(
   f"Accuracy with respect to the true energy: {np.abs(cx_noise[-1] - true_energy):.8f} Ha"
)

In [ ]:
## Energy vs. iteration plot
%matplotlib ipympl 
plt.figure(figsize=(10, 6))
plt.plot(range(niter_spsa + 1), simulation_energies, label = "Energy level of the current VQE using SPSA, NO noise")
plt.plot(range(niter_spsa + 1), random_noise_results, label = "Energy level of the current VQE using SPSA, RANDOM noise")
plt.plot(range(niter_spsa + 1), cx_noise, label = "Energy level of the current VQE using SPSA, CX depolarizing noise")
plt.title("H2 energy from the VQE using SPSA")
plt.xlabel("Number of optimization iterations", fontsize=14)
plt.ylabel("Energy (Ha)", fontsize=14)
plt.grid()
plt.hlines(true_energy, 0, niter_spsa+2, label="True energy level of H2", color = "red")
plt.legend(fontsize=14)
plt.show()

Homework: using the pennylane-qiskit plugin and a suitable backend, adapt the code above to be run on real hardware.